In [ ]:
# First, install the necessary library
!pip install youtube_transcript_api


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 11.1 MB/s eta 0:00:00


### METODE 1: YOUTUBE TRANSKRIP API TO TXT

In [ ]:
import whisper
import os
import re
from pytube import YouTube
import datetime

def get_video_id(url):
    """Extract video ID from YouTube URL"""
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None

def download_youtube_audio(url, output_path="audio"):
    """Download audio from YouTube video"""
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        # Create YouTube object
        yt = YouTube(url)
        
        # Get video details for file naming
        video_id = get_video_id(url)
        video_title = yt.title
        safe_title = re.sub(r'[^\w\-_]', '_', video_title)
        
        print(f"Downloading audio from: {video_title}")
        
        # Get audio stream and download
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(output_path=output_path)
        
        # Rename to mp3
        base, ext = os.path.splitext(output_file)
        new_file = f"{base}.mp3"
        os.rename(output_file, new_file)
        
        print(f"Audio downloaded to: {new_file}")
        return new_file, safe_title
        
    except Exception as e:
        print(f"Error downloading YouTube audio: {e}")
        return None, None

def transcribe_audio(audio_path, language="id"):
    """Transcribe audio using OpenAI Whisper"""
    try:
        print(f"Loading Whisper model...")
        model = whisper.load_model("medium")  # You can change model size: tiny, base, small, medium, large
        
        print(f"Transcribing audio... (this may take some time)")
        result = model.transcribe(audio_path, language=language)
        
        return result["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

# Main process
def process_youtube_speech(youtube_url, language="id"):
    """Process YouTube video: download audio and transcribe"""
    # Download audio
    audio_file, video_title = download_youtube_audio(youtube_url)
    
    if not audio_file:
        return
    
    # Transcribe audio
    transcription = transcribe_audio(audio_file, language=language)
    
    if not transcription:
        return
    
    # Save transcription to file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"Soekarno_{timestamp}.txt"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(f"Source: {youtube_url}\n")
        f.write(f"Title: {video_title}\n")
        f.write(f"Transcribed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(transcription)
    
    print(f"✓ Transcription complete! Saved to {output_filename}")
    print("\nPreview:")
    print(transcription[:500] + "..." if len(transcription) > 500 else transcription)
    
    return output_filename

# Example usage
youtube_url = "https://www.youtube.com/watch?v=N_Cp4KBrRZw"  # Change to your Soekarno speech video
process_youtube_speech(youtube_url, language="id")

Created output directory
Transcript:
[Music] The purpose of the United Nations should be to solve problems, to use it as a mere debating platform or as a propaganda outlet or as an extension of domestic politics is to pervert the high ideals which should imbue this body in order to modernize in order to modernize and make efficient our organization. Perhaps even the secretariat under the leadership of its secretary general. may need to be revised. In saying this, I am not most definitely not in any way criticizing or denouncing the present secretary general who is striving to do a good job under outmoded conditions which must at times seem impossible. How then can they be efficient? How can members of those two groups in the world, groups which are a reality and which must be accepted, how can members of those two groups feel at ease in this organization and have the necessary utmost confidence in it. Since the war, we have witnessed three great permanent phenomena. First is the rise o

### METHOD 2: OPEN AI WHISPER

In [1]:
# Install all required packages
!pip install openai-whisper ffmpeg-python pytube

In [3]:
!pip install yt-dlp

   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ------------------------- -------------- 2.1/3.3 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 3.3/3.3 MB 9.6 MB/s eta 0:00:00


In [4]:
# Option 1: Try to install FFmpeg via pip (simpler but may not always work)
!pip install ffmpeg-python

# Option 2: Try to install via conda if you're using Anaconda
# !conda install -c conda-forge ffmpeg

In [5]:
import whisper
import os
import re
from pytube import YouTube
import datetime
import yt_dlp

def get_video_id(url):
    """Extract video ID from YouTube URL"""
    video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if video_id_match:
        return video_id_match.group(1)
    return None


def download_with_ytdlp(url, output_path="audio", ffmpeg_location=None):
    """Alternative download using yt-dlp with ffmpeg handling"""
    try:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        video_id = get_video_id(url)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{output_path}/audio_{video_id}_{timestamp}.mp3"
        
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': output_file,
            'quiet': False,
            'no_warnings': False
        }
        
        # Add ffmpeg location if provided
        if ffmpeg_location:
            ydl_opts['ffmpeg_location'] = ffmpeg_location
            ydl_opts['postprocessors'] = [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }]
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            video_title = info.get('title', f"video_{video_id}")
            
        print(f"Audio downloaded to: {output_file}")
        return output_file, video_title
        
    except Exception as e:
        print(f"Error downloading with yt-dlp: {e}")
        return None, None

def download_youtube_audio(url, output_path="audio"):
    """Download audio from YouTube video using pytube"""
    try:
        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            
        # Create YouTube object
        yt = YouTube(url)
        
        # Get video details for file naming
        video_id = get_video_id(url)
        video_title = yt.title
        safe_title = re.sub(r'[^\w\-_]', '_', video_title)
        
        print(f"Downloading audio from: {video_title}")
        
        # Get audio stream and download
        audio_stream = yt.streams.filter(only_audio=True).first()
        output_file = audio_stream.download(output_path=output_path)
        
        # Rename to mp3
        base, ext = os.path.splitext(output_file)
        new_file = f"{base}.mp3"
        os.rename(output_file, new_file)
        
        print(f"Audio downloaded to: {new_file}")
        return new_file, safe_title
        
    except Exception as e:
        print(f"Error with pytube: {e}")
        return None, None
def transcribe_audio(audio_path, language="id"):
    """Transcribe audio using OpenAI Whisper"""
    try:
        print(f"Loading Whisper model...")
        model = whisper.load_model("medium")  # You can change model size: tiny, base, small, medium, large
        
        print(f"Transcribing audio... (this may take some time)")
        result = model.transcribe(audio_path, language=language)
        
        return result["text"]
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

def process_youtube_speech(youtube_url, language="id"):
    """Process YouTube video: download audio and transcribe"""
    # Try downloading with pytube first
    audio_file, video_title = download_youtube_audio(youtube_url)
    
    # If pytube fails, try yt-dlp
    if not audio_file:
        audio_file, video_title = download_with_ytdlp(youtube_url)
    
    if not audio_file:
        print("Failed to download audio using both methods.")
        return
    
    # Rest of the function remains the same
    # Transcribe audio
    transcription = transcribe_audio(audio_file, language=language)
    
    if not transcription:
        return
    
    # Save transcription to file
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"Soekarno_{timestamp}.txt"
    
    with open(output_filename, "w", encoding="utf-8") as f:
        f.write(f"Source: {youtube_url}\n")
        f.write(f"Title: {video_title}\n")
        f.write(f"Transcribed: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(transcription)
    
    print(f"✓ Transcription complete! Saved to {output_filename}")
    print("\nPreview:")
    print(transcription[:500] + "..." if len(transcription) > 500 else transcription)
    
    return output_filename

# Example usage
youtube_url = "https://www.youtube.com/watch?v=bcIk9n6nRUo&ab_channel=HendriTeja"  # Change to your Soekarno speech video
process_youtube_speech(youtube_url, language="id")

Error with pytube: HTTP Error 400: Bad Request
[youtube] Extracting URL: https://www.youtube.com/watch?v=bcIk9n6nRUo&ab_channel=HendriTeja
[youtube] bcIk9n6nRUo: Downloading webpage
[youtube] bcIk9n6nRUo: Downloading tv client config
[youtube] bcIk9n6nRUo: Downloading tv player API JSON
[youtube] bcIk9n6nRUo: Downloading ios player API JSON
[youtube] bcIk9n6nRUo: Downloading m3u8 information
[info] bcIk9n6nRUo: Downloading 1 format(s): 251
[download] Destination: audio\audio_bcIk9n6nRUo_20250523_220126.mp3
[download] 100% of    2.25MiB in 00:00:00 at 4.43MiB/s   
Audio downloaded to: audio/audio_bcIk9n6nRUo_20250523_220126.mp3
Loading Whisper model...


 20%|███████▋                              | 294M/1.42G [00:43<02:54, 7.01MiB/s]


KeyboardInterrupt: 